# WEEK 1

In [29]:
import math

In [30]:
# pip install watermark
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [31]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 3.7.3
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
statsmodels 0.9.0
sklearn 0.20.3

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores  : 8
interpreter: 64bit
Git hash   :


In [32]:
from __future__ import division, print_function
# turn off all Anaconda warnings
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

Let's look at one of the files with the data on the web pages visited by the user (number 31).

In [33]:
# change data path
PATH_TO_DATA = 'capstone_user_identification'

In [34]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '3users/user0003.csv'))

In [35]:
user31_data

,timestamp,site
0,2013-11-15 09:28:17,meduza.io
1,2013-11-15 09:33:04,google.com
2,2013-11-15 09:52:48,oracle.com
3,2013-11-15 11:37:26,google.com
4,2013-11-15 11:40:32,oracle.com
5,2013-11-15 11:40:34,google.com
6,2013-11-15 11:40:35,google.com
7,2013-11-15 11:40:37,mail.google.com
8,2013-11-15 11:40:40,yandex.ru
9,2013-11-15 11:41:35,meduza.io


### Prepare train data

In [36]:
import glob
import time
import collections
from tqdm.auto import tqdm
from collections import Counter
from glob import glob

In [37]:
def prepare_train_set(path_to_csv_files, session_length=10):
    
    stock_files = sorted(glob(path_to_csv_files))
    
    #create dataframe from file
    df = pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True)
    
    #sorted site
    sorted_site = dict(collections.OrderedDict(sorted(Counter(df.site).items(), key=lambda kv: kv[1], reverse = True)))
    #print(sorted_site)    
    sorted_site_list = list(sorted_site.keys())
    df_site_dict_2 = {}
    for i, site in enumerate(sorted_site, 1):
        df_site_dict_2[site] = (i, sorted_site.get(site))  
    
    #create number list site
    list_all_site = []
    user = 1
    for filename in tqdm((stock_files), desc='Loop2'):
        tmp_df = pd.read_csv(filename)
        list_site = []
        for site in tqdm(tmp_df.site, desc = 'Loop3'):
            list_site.append(df_site_dict_2.get(site)[0])
            #print()
        #print(len(list_site))
        #print(list_site)
        multiple_len =  (math.ceil(len(list_site)/session_length))*session_length
        tmp = [0] * (multiple_len - len(list_site))
        #tmp_little_list = list_site.copy()
        list_site.extend(tmp)
        #print(len(list_site))
        #print(list_site)
        count = 0
        while (count < (len(list_site)/10)):
            ind_1 = count * 10
            count = count + 1
            ind_2 = count * 10
            sess = list_site[ind_1 : ind_2]
            sess.append(user)
            #print(sess)
            list_all_site.append(sess)
            #print(dt_tmp)
        user = user + 1

    #create dataFrame
    name_site = []
    for i in range(session_length):
        name_site.append('site'+str(i+1))
    name_site.append('user_id')
    dt_tmp = pd.DataFrame(list_all_site, columns=name_site)
    
    return(dt_tmp, df_site_dict_2)

In [83]:
path = os.path.join(PATH_TO_DATA, '3users/*.csv')
session_length=10

train_data_toy, site_freq_3users = prepare_train_set(path)

In [39]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,3,2,2,7,2,1,8,5,9,10,1
1,3,1,1,1,0,0,0,0,0,0,1
2,3,2,6,6,2,0,0,0,0,0,2
3,4,1,2,1,2,1,1,5,11,4,3
4,4,1,2,0,0,0,0,0,0,0,3


In [26]:
tmp = csr_matrix(train_data_toy.iloc[:, :-1].values)

In [28]:
tmp.todense()

matrix([[ 3,  2,  2,  7,  2,  1,  8,  5,  9, 10],
        [ 3,  1,  1,  1,  0,  0,  0,  0,  0,  0],
        [ 3,  2,  6,  6,  2,  0,  0,  0,  0,  0],
        [ 4,  1,  2,  1,  2,  1,  1,  5, 11,  4],
        [ 4,  1,  2,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [11]:
train_data_toy

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,user_id
0,3,2,2,7,2,1,8,5,9,10,1
1,3,1,1,1,0,0,0,0,0,0,1
2,3,2,6,6,2,0,0,0,0,0,2
3,4,1,2,1,2,1,1,5,11,4,3
4,4,1,2,0,0,0,0,0,0,0,3


In [12]:
site_freq_3users

{'google.com': (1, 9),
 'oracle.com': (2, 8),
 'vk.com': (3, 3),
 'meduza.io': (4, 3),
 'mail.google.com': (5, 2),
 'football.kulichki.ru': (6, 2),
 'geo.mozilla.org': (7, 1),
 'accounts.google.com': (8, 1),
 'apis.google.com': (9, 1),
 'plus.google.com': (10, 1),
 'yandex.ru': (11, 1)}

In [81]:
path = os.path.join(PATH_TO_DATA, '10users/*.csv')

train_data_toy, site_freq_10users = prepare_train_set(path)

#### Часть 1. Сколько уникальных сессий из 10 сайтов в выборке с 10 пользователями?

In [14]:
train_data_toy.shape[0]

14061

#### Часть 2. Сколько всего уникальных сайтов в выборке из 10 пользователей?

In [15]:
len(site_freq_10users)

4913

#### Часть 3. Сколько уникальных сессий из 10 сайтов в выборке со 150 пользователями?

In [16]:
path = os.path.join(PATH_TO_DATA, '150users/*.csv')

In [17]:
%time
train_data_toy, site_freq_150users = prepare_train_set(path)

Wall time: 0 ns


In [18]:
train_data_toy.shape[0]

137019

#### Часть 4. Сколько всего уникальных сайтов в выборке из 150 пользователей?

In [19]:
len(site_freq_150users)

27797

#### Часть 5. Каковы топ-10 самых популярных сайтов среди посещенных 150 пользователями?

In [20]:
list(site_freq_150users.keys())[:10]

['www.google.fr',
 'www.google.com',
 'www.facebook.com',
 'apis.google.com',
 's.youtube.com',
 'clients1.google.com',
 'mail.google.com',
 'plus.google.com',
 'safebrowsing-cache.google.com',
 'www.youtube.com']

### Часть 1. Подготовка нескольких обучающих выборок для сравнения

In [80]:
# create zero array
from numpy import zeros

print(a)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [90]:
tmp = train_data_toy.iloc[:, :-1].values
a = zeros([tmp.shape[0],tmp.shape[1]])
for i in range(tmp.shape[0]):
    unique, counts = np.unique(tmp[i], return_counts=True)
    tmp_dict = dict(zip(unique, counts))
    for key, values in tmp_dict.items():
        if key == 0:
            continue
        else:
            pass
            a[i-1][key-1] = values
            print(i, " ", key, " ", values)
    #print(tmp_dict)

0   1   1
0   2   3
0   3   1
0   5   1
0   7   1
0   8   1
0   9   1
0   10   1
1   1   3
1   3   1
2   2   2
2   3   1
2   6   2
3   1   4
3   2   2
3   4   2
3   5   1


IndexError: index 10 is out of bounds for axis 0 with size 10

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy import stats
from statsmodels.stats.proportion import proportion_confint
%matplotlib inline
from matplotlib import pyplot as plt

In [7]:
def prepare_sparse_train_set_window(path_to_csv_files, session_length=10, window_size=10):
    stock_files = sorted(glob(path_to_csv_files))

    #create dataframe from file
    df = pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True)

    #sorted site
    sorted_site = dict(collections.OrderedDict(sorted(Counter(df.site).items(), key=lambda kv: kv[1], reverse = True)))
    #print(sorted_site)    
    sorted_site_list = list(sorted_site.keys())
    df_site_dict_2 = {}
    for i, site in tqdm(enumerate(sorted_site, 1), desc = 'Create sort site'):
        df_site_dict_2[site] = (i, sorted_site.get(site))  
    #print(df_site_dict_2)
    list_all_site = []
    
    #create dict site and session
    user = 1
    user_list = []
    for filename in tqdm(stock_files, desc = "List all sess"):
        tmp_df = pd.read_csv(filename)
        list_site = [0] * (session_length + 1)
        list_site_file = list(tmp_df.site)
        for i in tqdm(range(0, len(list(tmp_df.site)), window_size), desc = "Window size sess"):
            list_site_2 = [0] * (session_length)
            tmp_list = []        
            pos = 0
            for j in tqdm(range(i, session_length+i), desc = "Create one sess"):
                #print("POS = ", pos)
                #print("J = ", j)
                if j < len(list_site_file):
                    list_site_2[pos] = df_site_dict_2.get(list_site_file[j])[0]
                    pos = pos + 1
            #print(list_site_2)
            user_list.append(user)
            list_all_site.append(list_site_2)
        user = user + 1
    #print('RESULT')
    tmp_arr = np.zeros((len(list_all_site), len(sorted_site_list)))
    #print(user_list)
    
    #create array
    for i, sess in tqdm(enumerate(list_all_site), desc = 'Create array'):
        for j in range(0, tmp_arr.shape[1]):
            #print(sess)
            #print(sess.count(j))
            tmp_arr[i][j]=sess.count(j+1)
        
    return(tmp_arr, user_list)

NameError: name 'PATH_TO_DATA' is not defined

In [19]:
X_toy_s5_w3, y_s5_w3 = prepare_sparse_train_set_window(os.path.join(PATH_TO_DATA,'3users/*.csv'), session_length=5, window_size=3)

NameError: name 'prepare_sparse_train_set_window' is not defined

In [15]:
tmp = sparse.csr_matrix(X_toy_s5_w3)

NameError: name 'stock_files' is not defined

In [20]:
#stock_files = sorted(glob(path_to_csv_files))

#create dataframe from file
df = pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True)

#sorted site
sorted_site = dict(collections.OrderedDict(sorted(Counter(df.site).items(), key=lambda kv: kv[1], reverse = True)))
#print(sorted_site)    
sorted_site_list = list(sorted_site.keys())
df_site_dict_2 = {}
for i, site in enumerate(sorted_site, 1):
    df_site_dict_2[site] = (i, sorted_site.get(site))  

#create number list site
list_all_site = []
user = 1
for filename in tqdm((stock_files), desc='Loop2'):
    tmp_df = pd.read_csv(filename)
    list_site = []
    for site in tqdm(tmp_df.site, desc = 'Loop3'):
        list_site.append(df_site_dict_2.get(site)[0])
        #print()
    #print(len(list_site))
    #print(list_site)
    multiple_len =  (math.ceil(len(list_site)/session_length))*session_length
    tmp = [0] * (multiple_len - len(list_site))
    #tmp_little_list = list_site.copy()
    list_site.extend(tmp)
    #print(len(list_site))
    #print(list_site)
    count = 0
    while (count < (len(list_site)/10)):
        ind_1 = count * 10
        count = count + 1
        ind_2 = count * 10
        sess = list_site[ind_1 : ind_2]
        sess.append(user)
        #print(sess)
        list_all_site.append(sess)
        #print(dt_tmp)
    user = user + 1

In [32]:
list_all_site

[[3, 2, 2, 7, 2],
 [7, 2, 1, 8, 5],
 [8, 5, 9, 10, 3],
 [10, 3, 1, 1, 1],
 [1, 1, 0, 0, 0],
 [3, 2, 6, 6, 2],
 [6, 2, 0, 0, 0],
 [4, 1, 2, 1, 2],
 [1, 2, 1, 1, 5],
 [1, 5, 11, 4, 4],
 [4, 4, 1, 2, 0],
 [2, 0, 0, 0, 0]]

In [55]:
tmp_count = []
index = []
value = []
for i, s in enumerate(list_all_site):
    tmp_count.extend(Counter(s).keys())
    print([i] * 5)
    index.extend([i] * len(Counter(s).keys()))
    value.extend(Counter(s).keys())
    #print(Counter(s).keys())

[0, 0, 0, 0, 0]
[1, 1, 1, 1, 1]
[2, 2, 2, 2, 2]
[3, 3, 3, 3, 3]
[4, 4, 4, 4, 4]
[5, 5, 5, 5, 5]
[6, 6, 6, 6, 6]
[7, 7, 7, 7, 7]
[8, 8, 8, 8, 8]
[9, 9, 9, 9, 9]
[10, 10, 10, 10, 10]
[11, 11, 11, 11, 11]


In [56]:
tmp_count_arr = np.asarray(tmp_count)
index_arr = np.asarray(index)
value_arr = np.asarray(value)

In [66]:
value_arr

array([ 3,  2,  7,  7,  2,  1,  8,  5,  8,  5,  9, 10,  3, 10,  3,  1,  1,
        0,  3,  2,  6,  6,  2,  0,  4,  1,  2,  1,  2,  5,  1,  5, 11,  4,
        4,  1,  2,  0,  2,  0])

In [64]:
tmp = csr_matrix((value_arr, tmp_count_arr, index_arr))

In [76]:
import re
import itertools

In [89]:
count = 0
df = pd.DataFrame()

# make one dataframe
for row in stock_files:

    # pick up user id
    m = re.compile(".*user(.*).csv.*").match(row).groups()

    if len(m) > 0:
        userid = int(m[0])

        if count == 0:
            df = pd.read_csv(row)
            df['user_id'] = userid
        else:
            df_userid = pd.read_csv(row)
            df_userid['user_id'] = userid

            df = df.append(df_userid, ignore_index=True)
        count += 1
print(df)

              timestamp                  site  user_id
0   2013-11-15 09:28:17                vk.com        1
1   2013-11-15 09:33:04            oracle.com        1
2   2013-11-15 09:52:48            oracle.com        1
3   2013-11-15 11:37:26       geo.mozilla.org        1
4   2013-11-15 11:40:32            oracle.com        1
5   2013-11-15 11:40:34            google.com        1
6   2013-11-15 11:40:35   accounts.google.com        1
7   2013-11-15 11:40:37       mail.google.com        1
8   2013-11-15 11:40:40       apis.google.com        1
9   2013-11-15 11:41:35       plus.google.com        1
10  2013-11-15 12:40:35                vk.com        1
11  2013-11-15 12:40:37            google.com        1
12  2013-11-15 12:40:40            google.com        1
13  2013-11-15 12:41:35            google.com        1
14  2013-11-15 09:28:17                vk.com        2
15  2013-11-15 09:33:04            oracle.com        2
16  2013-11-15 09:52:48  football.kulichki.ru        2
17  2013-1

In [82]:
# make frequency dictionary
udf = df['site'].value_counts(sort=True)
#print(udf)

unique = {}
i = 0
for index, row in udf.iteritems():
    i += 1
    unique[index] = i, row

#print(unique)
n = session_length + 1

# make output dataframe
split = 1
out = []
line = np.zeros((1, n))
user_id = 0
prev_user_id: int = 0
as_matrix = df.as_matrix()
window_step = 0
i = 0
print(as_matrix)

while i < len(as_matrix):

    # get first user
    if user_id == 0:
        prev_user_id = as_matrix[i][2]

    user_id = as_matrix[i][2]
    site_id = unique[as_matrix[i][1]][0]

    line[0, split - 1] = site_id

    # check end or change user
    if user_id != prev_user_id:
        line[0, split - 1] = 0
        split = session_length

    if (i == (len(as_matrix) - 1)) and (window_step + window_size > (len(as_matrix) - 1)):
        split = session_length

    if split < session_length:
        i += 1

    else:  # limit

        if user_id != prev_user_id:
            line[0, n - 1] = prev_user_id
            prev_user_id = user_id
            window_step = i
        else:
            line[0, n - 1] = user_id
            window_step += window_size
            if i <= (len(as_matrix) - 1):
                i = window_step
            else:
                out.append(line)
                break

        out.append(line)
        line = np.zeros((1, n))
        split = 0

    split += 1

data = np.concatenate(out)

tmp_arr = get_sparse_matrice(data[:, :-1], data.shape[0], len(unique.keys())), data[:, -1]

[['2013-11-15 09:28:17' 'vk.com' 1]
 ['2013-11-15 09:33:04' 'oracle.com' 1]
 ['2013-11-15 09:52:48' 'oracle.com' 1]
 ['2013-11-15 11:37:26' 'geo.mozilla.org' 1]
 ['2013-11-15 11:40:32' 'oracle.com' 1]
 ['2013-11-15 11:40:34' 'google.com' 1]
 ['2013-11-15 11:40:35' 'accounts.google.com' 1]
 ['2013-11-15 11:40:37' 'mail.google.com' 1]
 ['2013-11-15 11:40:40' 'apis.google.com' 1]
 ['2013-11-15 11:41:35' 'plus.google.com' 1]
 ['2013-11-15 12:40:35' 'vk.com' 1]
 ['2013-11-15 12:40:37' 'google.com' 1]
 ['2013-11-15 12:40:40' 'google.com' 1]
 ['2013-11-15 12:41:35' 'google.com' 1]
 ['2013-11-15 09:28:17' 'vk.com' 2]
 ['2013-11-15 09:33:04' 'oracle.com' 2]
 ['2013-11-15 09:52:48' 'football.kulichki.ru' 2]
 ['2013-11-15 11:37:26' 'football.kulichki.ru' 2]
 ['2013-11-15 11:40:32' 'oracle.com' 2]
 ['2013-11-15 09:28:17' 'meduza.io' 3]
 ['2013-11-15 09:33:04' 'google.com' 3]
 ['2013-11-15 09:52:48' 'oracle.com' 3]
 ['2013-11-15 11:37:26' 'google.com' 3]
 ['2013-11-15 11:40:32' 'oracle.com' 3]
 ['2

In [90]:
def get_sparse_matrice(data, rownum, colnum):
    index = 0
    col = []
    row = []
    prepared_data = []

    for x in enumerate(data):
        nonzero = np.ma.masked_equal(x[1], 0).compressed()  # masked zero
        counted = np.unique(nonzero, return_counts=True)

        col_item = [xx - 1 for xx in counted[0]]
        row_item = [index] * len(col_item)
        data_item = counted[1]

        col.append(col_item)
        row.append(row_item)
        prepared_data.append(data_item)
        index += 1

    cols = np.array(list(itertools.chain.from_iterable(col)))
    rows = np.array(list(itertools.chain.from_iterable(row)))
    all_data = np.array(list(itertools.chain.from_iterable(prepared_data)))

    output = csr_matrix((all_data, (rows, cols)), shape=(rownum, colnum))

    return output

In [87]:
tmp_arr[0].todense()

matrix([[0, 3, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
        [3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0]], dtype=int64)